# Solving for power using MNA
Last update: 25 Jan 2026

This notebook attempts to answer a question posted on electronics stackexchange and linked here: [Solving for power using MNA](https://electronics.stackexchange.com/questions/764874/solving-for-power-using-mna)

I hope you are doing well. I am struggling to solve this modified nodal analysis question (MNA), but I think I am close. I have determined the 6 unknowns and the 6 equations needed to solve for all of the unknowns, but I must of messed up something in the equations, or perhaps my power calculation is wrong. Below is an image of the original circuit, the problem description, and my work. Thanks so much!

Problem: Using MNA as seen in class and in the slides, solve the circuit in the figure. All resistances are given in ohms. The value of R = 11 ohms. What is the power absorbed by the 7A source. If the source is absorbing power, report it as positive.

---

I'm presenting yet another answer to the question, only to illustrate that often there is more than one way to approach and solve a problem like this. I'll start by saying that my answer for the power is in agreement with what has already been offered, it's my procedure that is a bit different. I hope the community finds it useful.

To answer this question, I'll present some calculations that were performed using Python in a JupyterLab notebook; the link to my notebook on GitHub is [here](). What you see below are sections of the notebook converted to markdown and pasted into the answer dialog box. 

Since the problem schematic didn't include node numbers or reference designators for the components, I re-drew the schematic in LTSpice. 

![schematic](Solving_for_power_using_MNA.png)

In [1]:
from sympy import *
import numpy as np
from tabulate import tabulate
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
import SymMNA
from IPython.display import display, Markdown, Math, Latex
init_printing()

My solution starts by exporting the netlist from LTSpice and pasting the text into the cell below.

In [2]:
net_list = '''
V1 3 0 7
V2 6 4 10
R1 3 6 3
V3 5 3 5
R2 5 2 7
R3 2 7 11
R4 8 2 10
R5 0 4 3
R6 4 1 11
R7 0 1 11
I1 1 7 7
G1 8 0 2 4 3
R8 3 1 9
R9 4 2 3
'''

A [Modified Nodal Analysis](https://en.wikipedia.org/wiki/Modified_nodal_analysis) (MNA) procedure was implemented in Python. The function `SymMNA.smna(net_list)` returns the network matrices along with a report and two Pandas data frames. The Python module available on GitHub at [SymMNA](https://github.com/Tiburonboy/SymMNA). Several parameters are returned by the function, but only one of the dataframes and the matrices $A$, $X$ and $Z$ are used in the analysis below.

In [3]:
report, network_df, i_unk_df, A, X, Z = SymMNA.smna(net_list)

The code below assembles the network equations from the MNA matrices and displays the equations.

In [4]:
# Put matrices into SymPy 
X = Matrix(X)
Z = Matrix(Z)

NE_sym = Eq(A*X,Z)

# the free symbols are entered as SymPy variables and the element values are put into a dictionary.
var(str(NE_sym.free_symbols).replace('{','').replace('}',''))
element_values = SymMNA.get_part_values(network_df) # get element vales from netlist

# display the equations
temp = ''
for i in range(shape(NE_sym.lhs)[0]):
    temp += '${:s} = {:s}$<br>'.format(latex(NE_sym.rhs[i]),latex(NE_sym.lhs[i]))
Markdown(temp)

$- I_{1} = v_{1} \cdot \left(\frac{1}{R_{8}} + \frac{1}{R_{7}} + \frac{1}{R_{6}}\right) - \frac{v_{3}}{R_{8}} - \frac{v_{4}}{R_{6}}$<br>$0 = v_{2} \cdot \left(\frac{1}{R_{9}} + \frac{1}{R_{4}} + \frac{1}{R_{3}} + \frac{1}{R_{2}}\right) - \frac{v_{4}}{R_{9}} - \frac{v_{8}}{R_{4}} - \frac{v_{7}}{R_{3}} - \frac{v_{5}}{R_{2}}$<br>$0 = I_{V1} - I_{V3} + v_{3} \cdot \left(\frac{1}{R_{8}} + \frac{1}{R_{1}}\right) - \frac{v_{1}}{R_{8}} - \frac{v_{6}}{R_{1}}$<br>$0 = - I_{V2} + v_{4} \cdot \left(\frac{1}{R_{9}} + \frac{1}{R_{6}} + \frac{1}{R_{5}}\right) - \frac{v_{2}}{R_{9}} - \frac{v_{1}}{R_{6}}$<br>$0 = I_{V3} - \frac{v_{2}}{R_{2}} + \frac{v_{5}}{R_{2}}$<br>$0 = I_{V2} - \frac{v_{3}}{R_{1}} + \frac{v_{6}}{R_{1}}$<br>$I_{1} = - \frac{v_{2}}{R_{3}} + \frac{v_{7}}{R_{3}}$<br>$0 = - g_{1} v_{4} + v_{2} \left(g_{1} - \frac{1}{R_{4}}\right) + \frac{v_{8}}{R_{4}}$<br>$V_{1} = v_{3}$<br>$V_{2} = - v_{4} + v_{6}$<br>$V_{3} = - v_{3} + v_{5}$<br>

As you can see my Python procedure generates more equations than the other answers. This is a consequence of the algorithmic procedure. The benefit is that if the schematic is correct, the exported netlist will be correct and the network equations can easily and automatically generated with little effort.  

The network equations for the netlist can be solved symbolically with SymPy's `solve` function.

In [5]:
%%time
U_sym = solve(NE_sym,X)

CPU times: user 2min 53s, sys: 39.1 ms, total: 2min 53s
Wall time: 2min 53s


The symbolic solution took about three minutes on my i3 laptop. The symbolic solution by itself is not that informative, so I won't display the results. However, by substituting the element values for the reference designators, we can simplify the expression for the power absorbed (or delivered) by the 7A current source, which is $I_1$ in my schematic. If we keep the resistors designated as $R$ in the original problem schematic unassigned, we can obtain an expression for the power as a function of $R$. The SymPy `cancel()` function will combine all the terms.

In [6]:
R = symbols('R') # declare the symbol R

ans = ((U_sym[v7] - U_sym[v1])*element_values[I1]).subs({V1: 7, V2: 10, V3: 5, R1: 3, R2: 7, R3: R, R4: 10, R5: 3, R6: R, R7: R, I1: 7, g1: 3, R8: 9, R9: 3}).cancel()
#ans = ((U_sym[v1] - U_sym[v7])*element_values[I1]).subs({V1: 7, V2: 10, V3: 5, R1: 3, R2: 7, R3: R, R4: 10, R5: 3, R6: R, R7: R, I1: 7, g1: 3, R8: 9, R9: 3}).cancel()
Markdown('$P={:s}$'.format(latex(ans)))

$P=\frac{7301.0 R^{3} + 202930.0 R^{2} + 148932.0 R + 34587.0}{149.0 R^{2} + 2901.0 R + 1971.0}$

Now we have the power expressed as a function of $R$, so we can easily obtain answers for other values of $R$.

Notice the order of `U_sym[v7] - U_sym[v1]`.  

The question asks: If the source is absorbing power, report it as positive.

The convention used by most SPICE programs is that positive current flows into the positive terminal. The positive terminal is the one pointed to by the arrow in the symbol. 

If we look at the node voltages for nodes 2 and 7, the numerical solution below has $v_7=76.8$ volts and $v_2=-0.197$ volts. I would naturally think the positive direction of the current flowing through $R_3$ is from a higher voltage to a lower voltage, i.e. towards $I_1$. 

Looking at node 1, where $v_1=-22.1$, this voltage is also lower than $v_7$, so the current is flowing into the positive terminal of the 7A source, which by convention is a positive current. 

Keeping the convention in mind, the source is absorbing power since the current flow is opposite the direction of the arrow on the symbol. Therefore we will report the power as positive.


For $R=11$ we get:

In [7]:
print(f'When R = 11, the power absorbed by the 7A source = {ans.subs({R:11}):.1f}W')

When R = 11, the power absorbed by the 7A source = 692.4W


And picking another value such as, $R=3$ we get:

In [8]:
print(f'When R = 3, the power absorbed by the 7A source = {ans.subs({R:3}):.1f}W')

When R = 3, the power absorbed by the 7A source = 208.5W


For large networks the symbolic solution of the network equations is not that interesting and takes too long. A numerical solution is very fast. To find the numerical solution to the network equations we can substitute the component values into the symbolic network equations and display them.

In [9]:
NE = NE_sym.subs(element_values)

temp = ''
for i in range(shape(NE.lhs)[0]):
    temp += '${:s} = {:s}$<br>'.format(latex(NE.rhs[i]),latex(NE.lhs[i]))

Markdown(temp)

$-7.0 = 0.292929292929293 v_{1} - 0.111111111111111 v_{3} - 0.0909090909090909 v_{4}$<br>$0 = 0.667099567099567 v_{2} - 0.333333333333333 v_{4} - 0.142857142857143 v_{5} - 0.0909090909090909 v_{7} - 0.1 v_{8}$<br>$0 = I_{V1} - I_{V3} - 0.111111111111111 v_{1} + 0.444444444444444 v_{3} - 0.333333333333333 v_{6}$<br>$0 = - I_{V2} - 0.0909090909090909 v_{1} - 0.333333333333333 v_{2} + 0.757575757575758 v_{4}$<br>$0 = I_{V3} - 0.142857142857143 v_{2} + 0.142857142857143 v_{5}$<br>$0 = I_{V2} - 0.333333333333333 v_{3} + 0.333333333333333 v_{6}$<br>$7.0 = - 0.0909090909090909 v_{2} + 0.0909090909090909 v_{7}$<br>$0 = 2.9 v_{2} - 3.0 v_{4} + 0.1 v_{8}$<br>$7.0 = v_{3}$<br>$10.0 = - v_{4} + v_{6}$<br>$5.0 = - v_{3} + v_{5}$<br>

Using SymPy's `solve` function we can get the numerical solution for the voltages and currents.

In [10]:
%%time
U = solve(NE,X)

CPU times: user 189 ms, sys: 2 µs, total: 189 ms
Wall time: 189 ms


Display the numerical solution using six digits to the right of the decimal so that results can be compared to the solution obtained from LTSpice.

In [11]:
table_header = ['unknowns', 'values','units']
table_row = []

for name, value in U.items():
    if str(name)[0] == 'I':
        unit = 'device current'
    else:
        unit = 'voltage'
    table_row.append([str(name),float(value),str(unit)])

print(tabulate(table_row, headers=table_header,colalign = ('right','decimal','left'),
    tablefmt="simple",floatfmt=('5s','.6f','20s')))

  unknowns      values  units
----------  ----------  --------------
        v1  -22.116546  voltage
        v2   -0.197241  voltage
        v3    7.000000  voltage
        v4   -2.819980  voltage
        v5   12.000000  voltage
        v6    7.180020  voltage
        v7   76.802759  voltage
        v8  -78.879409  voltage
      I_V1   -4.917628  device current
      I_V2   -0.060007  device current
      I_V3   -1.742463  device current


The solution above matches the LTSpice results. 